# 1D Spectral analysis

In this tutorial, we present the basics steps for a 1D spectral analysis. The main aim is to perform a spectral analysis of a point-like source. A spectral analysis consist in stacking all events in the region of interest. Doing this we forget about the spatial information of the source. This is why the 1D analysis is best suited for point-like sources. 

To estimate the background, we will use off regions taken on the data using reflected regions. 

## 1. Crab Nebula

### 1.1 Full containment IRFs: H.E.S.S. data

We are going to analyse the data from the High Energy Stereoscopic System (H.E.S.S.) towards the Crab nebula.  

Let's start with some basic imports:

In [ ]:
from gammapy.data import DataStore
from gammapy.maps import MapAxis, RegionGeom, WcsGeom
from gammapy.makers import SpectrumDatasetMaker, ReflectedRegionsBackgroundMaker, SafeMaskMaker, WobbleRegionsFinder
from gammapy.datasets import SpectrumDataset, Datasets
from gammapy.modeling.models import PowerLawSpectralModel, SkyModel, ExpCutoffPowerLawSpectralModel, LogParabolaSpectralModel
from gammapy.modeling import Fit
from gammapy.estimators import FluxPointsEstimator
from gammapy.visualization import plot_spectrum_datasets_off_regions

from astropy.coordinates import SkyCoord
import astropy.units as u
from regions import CircleSkyRegion, PointSkyRegion

import matplotlib.pyplot as plt
import numpy as np

Create a DataStore instance using the following path to H.E.S.S. data: `$GAMMAPY_DATA/hess-dl3-dr1`.

In [ ]:
data_store = 

Get the Crab coordinates using astropy `SkyCoord`.

In [ ]:
target_position = 

To select the relevant runs, we can use the `obs_table` associated to the `DataStore`.

In [ ]:
obs_table = data_store.obs_table

Using `select_sky_circle` method of `obs_table`, select run that are at most at 2.2 deg from the Crab Nebula.

In [ ]:
obs_table = 

We can then get the observations using the `get_observations` method of `DataStore`, passing to it the observation ids of the selected `obs_table`.

In [ ]:
observations = 

In [ ]:
print(observations)

Define a energy axis using `MapAxis.from_energy_bounds` between 0.1 and 40 TeV. 

Then define a relevant true energy axis. Remember that the true energy axis should have more bins that the reco energy axis and over a wider range. 

In [ ]:
energy_axis = 

energy_axis_true = 


Define a region of interest:

In [ ]:
on_region = 

For the crab, we have to create a exclusion mask. 

In [ ]:
exclusion_region = CircleSkyRegion(
    center=SkyCoord(183.604, -8.708, unit="deg", frame="galactic"),
    radius=0.5 * u.deg,
)

Plot the exclusion mask:

In [ ]:
skydir = target_position.galactic
geom = WcsGeom.create(
    npix=(150, 150), binsz=0.05, skydir=skydir, proj="TAN", frame="icrs"
)

exclusion_mask = ~geom.region_mask([exclusion_region])
ax = exclusion_mask.plot()
ax.scatter(target_position.ra, target_position.dec, transform=ax.get_transform("icrs"), label="Crab Nebula")
plt.show()

Create a geometry using `RegionGeom`.

In [ ]:
geom = 

Create an empty `SpectrumDataset`.

In [ ]:
dataset_empty = 

Create the Maker instance that we are going to need for the data reduction.

`SpectrumDatasetMaker` will "fill" the dataset with reduces IRFs and counts.

`ReflectedRegionsBackgroundMaker` will get the off counts from reflected regions.

`SafeMaskMaker` will define the safe mask for each dataset.

In [ ]:
dataset_maker = SpectrumDatasetMaker(
    containment_correction=True, selection=["counts", "exposure", "edisp"]
)
bkg_maker = ReflectedRegionsBackgroundMaker(exclusion_mask=exclusion_mask)
safe_mask_maker = SafeMaskMaker(methods=["aeff-max"], aeff_percent=10)

Run the data reduction loop. 

In [ ]:
datasets = Datasets()

for obs, obs_id in zip(observations, observations.ids):
    dataset = dataset_maker.run(dataset_empty.copy(name=obs_id), obs)
    dataset_on_off = bkg_maker.run(dataset, obs)
    dataset_on_off = safe_mask_maker.run(dataset_on_off, obs)
    datasets.append(dataset_on_off)

print(datasets)

Info table:

In [ ]:
info_table = datasets.info_table(cumulative=True)

In [ ]:
info_table

Plot the reflected regions:

In [ ]:
plt.figure()
ax = exclusion_mask.plot()
on_region.to_pixel(ax.wcs).plot(ax=ax, edgecolor="k")
plot_spectrum_datasets_off_regions(ax=ax, datasets=datasets)
plt.show()

Stack the dataset using `stack_reduce`.

In [ ]:
stacked = 

Create a power-law using the `PowerLawSpectralModel`.

In [ ]:
spectral_model = 

Create a `SkyModel`.

In [ ]:
sky_model = 

Fit the model to the stacked dataset.

In [ ]:
stacked.models = 

In [ ]:
fit = Fit()
result = 

In [ ]:
print(result)

In [ ]:
print(stacked.models)

Print the residuals:

In [ ]:
stacked.plot_residuals_spectral(method="diff/sqrt(model)")

Extract flux points:

In [ ]:
fp = FluxPointsEstimator(
     energy_edges=energy_axis.edges, 
    source="Crab", 
    selection_optional="all"
).run(datasets=stacked)

Plot the model and flux points.

In [ ]:
sed_type = "e2dnde"
energy_bounds = [0.4, 40]*u.TeV

stacked.models[0].spectral_model.plot(energy_bounds=energy_bounds, sed_type=sed_type, label="Best fit")
stacked.models[0].spectral_model.plot_error(energy_bounds=energy_bounds, sed_type=sed_type)
fp.plot(sed_type=sed_type, label="Flux Points")

#### Model significance

Exercise: Is a power-law the best model to describe the data ? 

To answer this question, we propose to use a likelihood ratio test (see Jonathan lecture) test both a `LogParabolaSpectralModel` and an `ExpCutoffPowerLawSpectralModel`.

Are those two models nested models to a power-law respectively ?

Can we apply Wilk's theorem ? 

Knowing that the "total stat" of the fit result is $-2\ln(\mathcal{L})$, where $\mathcal{L}$ is the likelihood, compute the significance of the exponentially cutoff power-law and the log-parabola against t


### 1.2 Point-like IRFs

#### Redo the Crab nebulae analysis using "Point-like" observations from MAGIC

For that you will have to get the data from `$GAMMAPY_DATA/magic/rad_max/data/`. 

To get the observations, `required_irf="point-like"` has to be passed to `DataStore.get_observations`.

Also note that MAGIC energy threshold is lower that H.E.S.S.. You can start your dataset reco energy axis at 50 GeV. Of course, you will have to adapt you true energy in consequences.

Finally, using "point-like" IRFs, it is better to use Wobble regions for the off counts. You can define region_finder of `ReflectedRegionsBackgroundMaker` to `ReflectedRegionsBackgroundMaker`. 

## Open problem: RX J1713.7-3946 1D

This morning we did the analysis of `RX J1713.7-3946` using the 3D anamysis. Do the same analysis with the using a spectral analisys.

Questions to ask yourself: 
 - What is the size of the region of interest for this source ?
 - Do you need an exclusion mask ?
 - Is your off regions correctly measured ?
 - Conclude !